In [1]:
"""
Script to render the asset pricing table
"""

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from scipy.stats import ttest_1samp
from regtabletotext import prettify_result
import warnings
warnings.filterwarnings("ignore")

from environ.constants import (
    DEPENDENT_VARIABLES,
    PROCESSED_DATA_PATH,
    STABLE_DICT,
    TABLE_PATH,
)
from environ.process.asset_pricing.assetpricing_functions import reg_fama_macbeth, clean_weekly_panel, univariate_sort, double_sort 


In [2]:
# load factors
ff3 = pd.read_csv(PROCESSED_DATA_PATH/"FF3.csv") 
ltw3 = pd.read_csv(PROCESSED_DATA_PATH/"LTW3.csv")

# load the regression panel dataset
reg_panel = pd.read_pickle(
    PROCESSED_DATA_PATH / "panel_main.pickle.zip", compression="zip"
)

# stable non-stable info dict
stable_nonstable_info = {
    "stablecoin": reg_panel[reg_panel["Token"].isin(STABLE_DICT.keys())],
    "non-stablecoin": reg_panel[~reg_panel["Token"].isin(STABLE_DICT.keys())],
    "all": reg_panel,
}

# How are returns aggregated for each portfolio
ret_agg = 'mean'

DEPENDENT_VARIABLES_bis = ['volume_ultimate_share']
# ,'eigen_centrality_undirected','total_eigen_centrality_undirected','Volume_share']

### Univariate sorting

In [3]:
for dom_variable in DEPENDENT_VARIABLES_bis:
    for is_boom in [-1]:
        n_quantiles = 5
        df_panel = clean_weekly_panel(reg_panel, dom_variable, is_stablecoin = 0, is_boom = is_boom)
        df_panel = pd.merge(df_panel,ff3, on='WeekYear')
        summary_table = univariate_sort(df_panel,dom_variable, n_quantiles, ret_agg = ret_agg)

        # Compute the difference: H minus L
        pivot = df_panel.pivot_table(index='WeekYear', columns='portfolio', values='ret')
        diff_returns = pivot[f'P{n_quantiles}'] - pivot['P1']
        mean_diff = diff_returns.mean() if ret_agg == 'mean' else diff_returns.median()
        std_diff = diff_returns.std(ddof=1)
        t_stat_diff, p_value_diff = ttest_1samp(diff_returns, popmean=0)

        summary_table[f'P{n_quantiles}-P1'] = {
            'Mean': mean_diff,
            't-Stat': t_stat_diff,
            # 'p-value': p_value_diff,
            'StdDev': std_diff,
            'Sharpe':  np.sqrt(365/7) * mean_diff / std_diff
        }
        if is_boom == 1:
            boom_str = " boom"
        elif is_boom == 0:
            boom_str = " bust"
        else:
            boom_str = " alltime"
        summary_table = summary_table.style.set_caption(dom_variable + boom_str)
        display(summary_table)

,P1,P2,P3,P4,P5,P5-P1
Mean,-0.024906,0.005745,0.030016,0.058223,0.070008,0.088555
t-Stat,-3.256858,0.744190,3.451940,5.654734,6.912133,5.622825
StdDev,0.245548,0.237051,0.268296,0.316184,0.319964,0.182989
Sharpe,-0.732431,0.174995,0.807871,1.329700,1.579953,3.494501


In [4]:
test = clean_weekly_panel(reg_panel, 'volume_ultimate_share', is_stablecoin = 0, is_boom = -1)
print(test.Token.nunique())
test.Token.value_counts(ascending=True).head(5)

102


Token
YFL      17
BTT      18
EUROC    18
DERC     18
LEND     18
Name: count, dtype: int64

### Asset pricing factor tests

In [5]:

for dom_variable in DEPENDENT_VARIABLES_bis:
    for is_boom in [-1]:
        n_quantiles = 3
        df_panel = clean_weekly_panel(reg_panel, dom_variable, is_stablecoin = 0, is_boom = is_boom)
        df_panel = pd.merge(df_panel,ff3, on='WeekYear')
        summary_table = double_sort(df_panel,dom_variable, secondary_variable='mcap', n_quantiles=n_quantiles, ret_agg=ret_agg)
        
        if is_boom == 1:
            boom_str = " boom"
        elif is_boom == 0:
            boom_str = " bust"
        else:
            boom_str = "alltime"
        summary_table = summary_table.style.set_caption(dom_variable + boom_str)
        display(summary_table)

primary_portfolio,P1,P2,P3
secondary_portfolio,,,
Q1,-0.024116,0.057274,0.115512
Q2,-0.015028,0.019835,0.070956
Q3,-0.008690,0.006596,0.025257


In [6]:
import statsmodels.api as sm
for dom_variable in DEPENDENT_VARIABLES_bis:
    for is_boom in [-1]:
        n_quantiles = 3
        df_panel = clean_weekly_panel(reg_panel, dom_variable, is_stablecoin = 0, is_boom = is_boom)
        summary_table = univariate_sort(df_panel,dom_variable, n_quantiles)

        # Compute the difference: H minus L
        pivot = df_panel.pivot_table(index='WeekYear', columns='portfolio', values='ret')
        pivot["CDOM"] = pivot[f"P{n_quantiles}"] - pivot['P1']
        dominance_factor = pivot[["CDOM"]].reset_index()
        assets_panel = clean_weekly_panel(reg_panel, dom_variable, is_stablecoin = 0, is_boom = -1)
        # merge all factors
        factors_data = pd.merge(dominance_factor, ff3, on=["WeekYear"], how="left")
        factors_data = pd.merge(factors_data, ltw3, on=["WeekYear"], how="left")
        # merge factors with returns
        factors_data = pd.merge(factors_data, assets_panel, on=["WeekYear"], how="left")
        factors_data = factors_data.dropna()
        # AP_test_ff3 = smf.ols(formula= 'CDOM ~ MKT + SMB + HML', data=factors_data).fit()
        # prettify_result(AP_test_ff3)
        AP_test_cmkt = smf.ols(formula= 'CDOM ~ CMKT', data=factors_data).fit()
        prettify_result(AP_test_cmkt)
        AP_test_ltw3 = smf.ols(formula= 'CDOM ~ CMKT + CMOM + CSIZE', data=factors_data).fit()
        prettify_result(AP_test_ltw3)

OLS Model:
CDOM ~ CMKT

Coefficients:
           Estimate  Std. Error  t-Statistic  p-Value
Intercept     0.085       0.002       43.192    0.000
CMKT          0.063       0.022        2.926    0.003

Summary statistics:
- Number of observations: 4,867
- R-squared: 0.002, Adjusted R-squared: 0.002
- F-statistic: 8.559 on 1 and 4865 DF, p-value: 0.003

OLS Model:
CDOM ~ CMKT + CMOM + CSIZE

Coefficients:
           Estimate  Std. Error  t-Statistic  p-Value
Intercept     0.082       0.002       40.081    0.000
CMKT          0.068       0.022        3.132    0.002
CMOM          0.087       0.030        2.918    0.004
CSIZE         0.092       0.032        2.851    0.004

Summary statistics:
- Number of observations: 4,867
- R-squared: 0.005, Adjusted R-squared: 0.005
- F-statistic: 8.444 on 3 and 4863 DF, p-value: 0.000



### FAMA MCBETH 

In [7]:
for dom_variable in DEPENDENT_VARIABLES_bis:
    for is_boom in [-1]:
        n_quantiles = 3
        df_panel = clean_weekly_panel(reg_panel, dom_variable, is_stablecoin = 0, is_boom = is_boom)
        summary_table = univariate_sort(df_panel,dom_variable, n_quantiles)

        # Compute the difference: H minus L
        pivot = df_panel.pivot_table(index='WeekYear', columns='portfolio', values='ret')
        pivot["CDOM"] = pivot[f"P{n_quantiles}"] - pivot['P1']
        dominance_factor = pivot[["CDOM"]].reset_index()
        assets_panel = clean_weekly_panel(reg_panel, dom_variable, is_stablecoin = 0, is_boom = -1)
        # merge all factors
        data_fama_macbeth = pd.merge(dominance_factor, ff3, on=["WeekYear"], how="left")
        data_fama_macbeth = pd.merge(data_fama_macbeth, ltw3, on=["WeekYear"], how="left")
        # merge factors with returns
        data_fama_macbeth = pd.merge(data_fama_macbeth, assets_panel, on=["WeekYear"], how="left")
        data_fama_macbeth = data_fama_macbeth.dropna()

        # run the Fama-Macbeth regression
        data_fama_macbeth['excess_ret'] = data_fama_macbeth['ret'] - data_fama_macbeth['RF']
        fama_macbeth = reg_fama_macbeth(data_fama_macbeth, formula="excess_ret ~ CMKT + CMOM + CSIZE + CDOM")
        if is_boom == 1:
            boom_str = " boom"
        elif is_boom == 0:
            boom_str = " bust"
        else:
            boom_str = " alltime"
        fama_macbeth = fama_macbeth.style.set_caption(dom_variable + boom_str)
        display(fama_macbeth)

,factor,risk_premium,t_stat,t_stat_NW
0,CDOM,0.876000,2.863000,2.448000
1,CMKT,0.905000,5.600000,5.808000
2,CMOM,0.044000,0.472000,0.436000
3,CSIZE,0.287000,3.252000,3.018000
4,Intercept,3.348000,2.583000,2.200000
